# 🎉 Attractor State With Serotonin Simulation

---
## 🔍 Notebook objectives

This notebook contains simulation of attractor state in the ring attractor network which contains 360 excitatory RS neurons and 90 inhibitory FS interneurons for different serotonin levels.

# 🎒 Setup

## ⬇️ Imports

In [ ]:
from utils.main_simulation import network_simulation_run
from utils.poisson_spike_train_generation import generate_spike_train
from utils.plots import *

import numpy as np
from scipy import stats
import yaml #reading env consts

## 🛠️ Simulation Utils

In [ ]:
#for calculating stopping time
def calculate_instantaneous_firing_rate(firings, window_size=5000, threshold_hz = 12, burst_threshold_hz = 80):
    num_time_points, num_of_neurons = firings.shape
    half_window = window_size // 2
    firing_rates = np.zeros(num_time_points)

    for t in range(half_window, num_time_points - half_window, 500):
        window_start = t - half_window
        window_end = t + half_window
        window_spikes = firings[window_start:window_end + 1, :]
        total_spikes = np.sum(window_spikes, axis=0)
        firing_rate = total_spikes * 10000 / window_size
        if np.any(firing_rate > burst_threshold_hz):
            return t / 10000
        elif np.any(firing_rate > threshold_hz):
            continue
        else:
            return t / 10000
    return num_time_points / 10000

In [ ]:
#RS neuron parameters
with open('utils/RS.yaml', 'r', encoding="utf-8") as f:
    params_RS = yaml.load(f, Loader=yaml.FullLoader)

#FS neuron parameters
with open('utils/FS.yaml', 'r', encoding="utf-8") as f:
    params_FS = yaml.load(f, Loader=yaml.FullLoader)

#receptor kinetics parameters
with open('utils/receptor_kinetics.yaml', 'r', encoding="utf-8") as f:
    params_receptor_kinetics = yaml.load(f, Loader=yaml.FullLoader)

#synaptic weights parameters
with open('utils/synaptic_weights.yaml', 'r', encoding="utf-8") as f:
    params_synaptic_weights = yaml.load(f, Loader=yaml.FullLoader)

#external input parameters
with open('utils/external_input.yaml', 'r', encoding="utf-8") as f:
    params_external_input = yaml.load(f, Loader=yaml.FullLoader)

#stimuli input parameters
with open('utils/stimuli_input.yaml', 'r', encoding="utf-8") as f:
    params_stimuli_input = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
#simulation time; always 5 seconds, for all experiments
seconds = 5

t_min = 0
t_max = int(seconds*1000) #in ms -> 1(s) of simulation
delta_T = 0.1 #0.1 ms is integration step
sim_steps = int(seconds*1000/delta_T)

T = np.linspace(t_min, t_max, sim_steps)

# 🧪 Experiments

## 1️⃣ Random Background Input Without Any Attractor

### 🏋️ Synaptic Weights

Definition of synaptic weights.

In [ ]:
#360 RS and 90 FS
Ne = 360
Ni = 90

weights = np.array(np.zeros((Ne + Ni, Ne + Ni))) #matrix (Ne + Ni) x (Ne + Ni) (i, j: i -> j), thus sum by column for input of j

# e -> e
for i in range(Ne):
    for j in range(Ne):
        angle_i = i / Ne * 360
        angle_j = j / Ne * 360
        weights[i, j] = params_synaptic_weights['j_ee'] * np.exp(-(min(max(angle_i, angle_j) - min(angle_i, angle_j), 360 - (max(angle_i, angle_j) - min(angle_i, angle_j))))**2/params_synaptic_weights['sigma']**2)

# e -> i
for i in range(Ne):
    for j in range(Ni):
        angle_i = i / Ne * 360
        angle_j = j / Ni * 360
        weights[i, Ne + j] = params_synaptic_weights['j_ei']

# i -> e
for i in range(Ni):
    for j in range(Ne):
        angle_i = i / Ni * 360
        angle_j = j / Ne * 360
        weights[Ne + i, j] = params_synaptic_weights['j_ie']
        
# i -> i
for i in range(Ni):
    for j in range(Ni):
        angle_i = i / Ni * 360
        angle_j = j / Ni * 360
        weights[Ne + i, Ne + j] = params_synaptic_weights['j_ii']

### 🚝 External Cortical Input & 🎯 Stimulus Representation

In [ ]:
v = np.array(np.zeros((len(T), Ne + Ni)))
v[:, :Ne] = params_external_input['v_e']
v[:, Ne:] = params_external_input['v_i']

eta = params_stimuli_input["eta"]
A = params_stimuli_input["A"]

t_stim_start = 1
t_stim_end = 1.25
delay_time = 3.75

### 🥳 Serotonin Weights

Define the set of parameter $\mu$ values.

In [ ]:
mus = [0.97, 0.98, 0.99, 1, 1.01, 1.02, 1.03]

### 🔩 Settings

In [ ]:
start_state = np.column_stack((np.append(params_RS["v"]*np.ones(Ne), params_FS["v"]*np.ones(Ni)), np.append(params_RS["u"]*np.ones(Ne), params_FS["u"]*np.ones(Ni)))) #matrix (Ne + Ni) x 2 - (v, u) for each neuron

params_network = {"a": np.append(params_RS["a"]*np.ones(Ne), params_FS["a"]*np.ones(Ni)), 
          "b": np.append(params_RS["b"]*np.ones(Ne), params_FS["b"]*np.ones(Ni)), 
          "c": np.append(params_RS["c"]*np.ones(Ne), params_FS["c"]*np.ones(Ni)), 
          "d": np.append(params_RS["d"]*np.ones(Ne), params_FS["d"]*np.ones(Ni)), 
          "C": np.append(params_RS["C"]*np.ones(Ne), params_FS["C"]*np.ones(Ni)), 
          "k": np.append(params_RS["k"]*np.ones(Ne), params_FS["k"]*np.ones(Ni)),
          "v_peak": np.append(params_RS["v_peak"]*np.ones(Ne), params_FS["v_peak"]*np.ones(Ni)), 
          "v_r": np.append(params_RS["v_r"]*np.ones(Ne), params_FS["v_r"]*np.ones(Ni)), 
          "v_t": np.append(params_RS["v_t"]*np.ones(Ne), params_FS["v_t"]*np.ones(Ni)),
          "tau_ampa": params_receptor_kinetics["tau_ampa"]*np.ones((Ne + Ni)),
          "tau_nmda": params_receptor_kinetics["tau_nmda"]*np.ones((Ne + Ni)),
          "tau_gabaa": params_receptor_kinetics["tau_gabaa"]*np.ones((Ne + Ni)),
          "tau_gabab": params_receptor_kinetics["tau_gabab"]*np.ones((Ne + Ni)),
          "g_ampa": params_receptor_kinetics["g_ampa"]*np.ones((Ne + Ni)),
          "g_nmda": params_receptor_kinetics["g_nmda"]*np.ones((Ne + Ni)),
          "g_gabaa": params_receptor_kinetics["g_gabaa"]*np.ones((Ne + Ni)),
          "g_gabab": params_receptor_kinetics["g_gabab"]*np.ones((Ne + Ni)),
          "g_e_external": params_external_input["g_e_external"]*np.ones((Ne + Ni)),
          "g_i_external": params_external_input["g_i_external"]*np.ones((Ne + Ni))
}

### 🏃 Run

The total completion of the simulation will take approximately 2 hours.

In [ ]:
sim_num = 30 #30 - default

In [ ]:
np.random.seed(42) #for reproducibility

mu_exc_firing_rates = {}
mu_inh_firing_rates = {}
mu_prediction_errors = {}
mu_attractor_state_terminations = {}

for mu in mus:
    exc_firing_rates = []
    inh_firing_rates = []
    prediction_errors = []
    attractor_state_terminations = []

    #prepare serotonin weights
    serotonin_weights = np.ones(Ne + Ni)
    serotonin_weights[:Ne] = mu*serotonin_weights[:Ne] 

    for _ in range(sim_num):

        #random selection of stimuli orientation
        stimuli_orientation = np.random.choice(np.arange(360))

        #present stimuli to the network
        for i in range(Ne):
            angle_i = i / Ne * 360
            v[int(t_stim_start*len(T)/seconds):int(t_stim_end*len(T)/seconds), i] = params_external_input['v_e'] * (1 + A * np.exp(eta * (np.cos(np.deg2rad(angle_i) - np.deg2rad(stimuli_orientation)) - 1)))
        spike_trains = generate_spike_train(Ne, Ni, v, T, delta_T)

        #simulation itself
        _, firings, _, _, _, _, _ = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, spike_trains, serotonin_weights, params_network)
    
        #observe firing rates only for the delay time (after stimuli presentation)
        observed_firing_rates = np.sum(firings[int(t_stim_end * 1000 / delta_T):], axis = 0) / delay_time

        #calculate predicted angle and error for prediction
        predicted_angle = np.dot(observed_firing_rates[:Ne] * delay_time, np.arange(Ne).T)/np.sum(observed_firing_rates[:Ne] * delay_time)
        prediction_errors.append(stimuli_orientation - predicted_angle)

        #excitatory firing rate calculation for 15 degrees window around stimuli
        exc_start_idx = (stimuli_orientation - 15) % Ne
        exc_end_idx = (stimuli_orientation + 15) % Ne
        if exc_start_idx <= exc_end_idx:
            exc_firing_rates += list(observed_firing_rates[exc_start_idx:exc_end_idx + 1])
        else:
            exc_firing_rates += list(np.concatenate((observed_firing_rates[exc_start_idx:], observed_firing_rates[:exc_end_idx + 1])))

        #inhibitory firing rate
        inh_firing_rates += list(observed_firing_rates[Ne:])

        #termination of attractor state
        attractor_state_terminations.append(calculate_instantaneous_firing_rate(firings[int(t_stim_end * 1000 / delta_T):, :360], window_size = 5000))
        
    mu_exc_firing_rates[mu] = exc_firing_rates
    mu_inh_firing_rates[mu] = inh_firing_rates
    mu_prediction_errors[mu] = prediction_errors
    mu_attractor_state_terminations[mu] = attractor_state_terminations

### 📈 Metrics

Mean firing rates for inhibitory neurons and excitatory ones which are around stimuli.

In [ ]:
for mu in mus:
    print(f"Mean value for excitatory firing rate for mu = {mu} is: {np.mean(mu_exc_firing_rates[mu]):.02f} (Hz).")
    print(f"Mean value for inhibitory firing rate for mu = {mu} is: {np.mean(mu_inh_firing_rates[mu]):.02f} (Hz).")
    print()

To reproduce Figure 5.8 (part 1).

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

data = []
for mu in mus[:4]:
    trace = go.Box(y=np.abs(mu_exc_firing_rates[mu]), name=f"Excitatory Firing Rates for mu = {mu}")
    data.append(trace)

layout = go.Layout(
    title='Attractor State Activity: Excitatory Neurons',
    yaxis=dict(title='Firing Rate (in Hz)')
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(height = 600, showlegend = False)
fig.show()

To reproduce Figure 5.8 (part 2).

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

data = []
for mu in mus[:4]:
    trace = go.Box(y=np.abs(mu_inh_firing_rates[mu]), name=f"Inhibitory Firing Rates for mu = {mu}")
    data.append(trace)

layout = go.Layout(
    title='Attractor State Activity: Inhibitory Neurons',
    yaxis=dict(title='Firing Rate (in Hz)')
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(height = 600, showlegend = False)
fig.show()

To reproduce Figure 5.11 (part 1).

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

data = []
for mu in mus[3:]:
    trace = go.Box(y=np.abs(mu_inh_firing_rates[mu]), name=f"Inhibitory Firing Rates for mu = {mu}")
    data.append(trace)

layout = go.Layout(
    title='Attractor State Activity',
    yaxis=dict(title='Firing Rate (in Hz)')
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(height = 600, showlegend = False)
fig.show()

To reproduce Figure 5.11 (part 2).

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

data = []
for mu in mus[3:]:
    trace = go.Box(y=np.abs(mu_exc_firing_rates[mu]), name=f"Excitatory Firing Rates for mu = {mu}")
    data.append(trace)

layout = go.Layout(
    title='Attractor State Activity',
    yaxis=dict(title='Firing Rate (in Hz)')
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(height = 600, showlegend = False)
fig.show()

Welch's test for equality of mean firing rates.

In [ ]:
print("Excitatory firing rates")
for mu in mus:
    if mu != 1:
        t_statistic, p_value = stats.ttest_ind(mu_exc_firing_rates[mu], mu_exc_firing_rates[1], equal_var=False)
        print(f"For mu = {mu}, p_value = {p_value:.04f}")   
        print()
print("Inhibitory firing rates")
for mu in mus:
    if mu != 1:
        t_statistic, p_value = stats.ttest_ind(mu_inh_firing_rates[mu], mu_inh_firing_rates[1], equal_var=False)
        print(f"For mu = {mu}, p_value = {p_value:.04f}")   
        print()

The end time of the attractor state. To reproduce Figure 5.9.

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

data = []
for mu in mus[:4]:
    trace = go.Box(y=np.abs(mu_attractor_state_terminations[mu]), name=f"mu = {mu}")
    data.append(trace)

layout = go.Layout(
    title='Attractor State Duration',
    yaxis=dict(title='Time (in s)')
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(height = 600, showlegend = False)
fig.show()

We didn't include this plot in the thesis as it is visually non-infromative but mentioned in the text that for $\mu = 1$ and for $\mu = 1.01$ all simulations were successful while for $\mu = 1.02$ and $\mu = 1.03$ network exhibits overstimulation right away.

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

data = []
for mu in mus[3:]:
    trace = go.Box(y=np.abs(mu_attractor_state_terminations[mu]), name=f"Attractor State Duration for mu = {mu}")
    data.append(trace)

layout = go.Layout(
    title='Attractor State Termination',
    yaxis=dict(title='Time (in s)')
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(height = 600, showlegend = False)
fig.show()

Prediction error assessment. To reproduce Figure 5.7.

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

data = []
for mu in mus:
    trace = go.Box(y=np.minimum(360 - np.abs(mu_prediction_errors[mu]), np.abs(mu_prediction_errors[mu])) , name=f"Prediction Error for mu = {mu}")
    data.append(trace)

layout = go.Layout(
    title='Prediction Error',
    yaxis=dict(title='Error (in degrees)')
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(height = 600, showlegend = False)
fig.show()

Welch's t-test for quality of mean prediction errors.

In [ ]:
for mu in mus:
    if mu != 1:
        t_statistic, p_value = stats.ttest_ind(np.minimum(360 - np.abs(mu_prediction_errors[mu]), np.abs(mu_prediction_errors[mu])), np.minimum(360 - np.abs(mu_prediction_errors[1]), np.abs(mu_prediction_errors[1])), equal_var=False)
        print(f"For mu = {mu}, p_value = {p_value:.04f}")   
        print()

### 👁️ Visualizations for Particular Run

Increased level of serotonin, $\mu = 0.97$.

In [ ]:
np.random.seed(42) #for reproducibility

#random selection of stimuli orientation
stimuli_orientation = 100
serotonin_weights = np.ones(Ne + Ni)
serotonin_weights[:Ne] = 0.97*serotonin_weights[:Ne] 

#present stimuli to the network
for i in range(Ne):
    angle_i = i / Ne * 360
    v[int(t_stim_start*len(T)/seconds):int(t_stim_end*len(T)/seconds), i] = params_external_input['v_e'] * (1 + A * np.exp(eta * (np.cos(np.deg2rad(angle_i) - np.deg2rad(stimuli_orientation)) - 1)))
spike_trains = generate_spike_train(Ne, Ni, v, T, delta_T)

#simulation itself
states, firings, synaptic_input, synaptic_inputs, background_input, conductances, background_conductance = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, spike_trains, serotonin_weights, params_network)

termination_time = calculate_instantaneous_firing_rate(firings[int(t_stim_end * 1000 / delta_T):, :360], window_size = 5000)
print(f"Termination time for attractor state is: {termination_time:.02f}")

To reproduce Figure 5.10.

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

fig = exc_inh_firing_rates(firings[:, :360], firings[:, 360:], [t_stim_start], [t_stim_end], delta_T)
fig.update_layout(height = 600)
fig.add_vline(x=termination_time + 1.25,
              line=dict(color='black', width=3),
              opacity = 0.7
              )
fig.show()

Decreased level of serotonin, $\mu = 1.01$.

In [ ]:
np.random.seed(42) #for reproducibility

#random selection of stimuli orientation
stimuli_orientation = 100
serotonin_weights = np.ones(Ne + Ni)
serotonin_weights[:Ne] = 1.01*serotonin_weights[:Ne] 

#present stimuli to the network
for i in range(Ne):
    angle_i = i / Ne * 360
    v[int(t_stim_start*len(T)/seconds):int(t_stim_end*len(T)/seconds), i] = params_external_input['v_e'] * (1 + A * np.exp(eta * (np.cos(np.deg2rad(angle_i) - np.deg2rad(stimuli_orientation)) - 1)))
spike_trains = generate_spike_train(Ne, Ni, v, T, delta_T)

#simulation itself
states, firings, synaptic_input, synaptic_inputs, background_input, conductances, background_conductance = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, spike_trains, serotonin_weights, params_network)

termination_time = calculate_instantaneous_firing_rate(firings[int(t_stim_end * 1000 / delta_T):, :360], window_size = 5000)
print(f"Termination time for attractor state is: {termination_time:.02f}")

To reproduce Figure 5.12.

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

fig = exc_inh_firing_rates(firings[:, :360], firings[:, 360:], [t_stim_start], [t_stim_end], delta_T)
fig.update_layout(height = 600)
fig.add_vline(x=termination_time + 1.25,
              line=dict(color='black', width=3),
              opacity = 0.7
              )
fig.show()

Decreased level of serotonin, $\mu = 1.02$.

In [ ]:
np.random.seed(42) #for reproducibility

#random selection of stimuli orientation
stimuli_orientation = 100
serotonin_weights = np.ones(Ne + Ni)
serotonin_weights[:Ne] = 1.01*serotonin_weights[:Ne] 

#present stimuli to the network
for i in range(Ne):
    angle_i = i / Ne * 360
    v[int(t_stim_start*len(T)/seconds):int(t_stim_end*len(T)/seconds), i] = params_external_input['v_e'] * (1 + A * np.exp(eta * (np.cos(np.deg2rad(angle_i) - np.deg2rad(stimuli_orientation)) - 1)))
spike_trains = generate_spike_train(Ne, Ni, v, T, delta_T)

#simulation itself
states, firings, synaptic_input, synaptic_inputs, background_input, conductances, background_conductance = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, spike_trains, serotonin_weights, params_network)

termination_time = calculate_instantaneous_firing_rate(firings[int(t_stim_end * 1000 / delta_T):, :360], window_size = 5000)
print(f"Termination time for attractor state is: {termination_time:.02f}")

To reproduce Figure 5.13.

In [ ]:
# @markdown Make sure you execute this cell to observe the plot

fig = exc_inh_firing_rates(firings[:, :360], firings[:, 360:], [t_stim_start], [t_stim_end], delta_T)
fig.update_layout(height = 600)
fig.add_vline(x=termination_time + 1.25,
              line=dict(color='black', width=3),
              opacity = 0.7
              )
fig.show()

### ⚡️ Currents Comparison

In [ ]:
synpaptic_inputs = {}
nmda_synaptic_inputs = {}
for mu in mus:
    np.random.seed(42) #for reproducibility; the only source of randomness in the model

    #random selection of stimuli orientation
    stimuli_orientation = 100
    serotonin_weights = np.ones(Ne + Ni)
    serotonin_weights[:Ne] = mu*serotonin_weights[:Ne] 

    #present stimuli to the network
    for i in range(Ne):
        angle_i = i / Ne * 360
        v[int(t_stim_start*len(T)/seconds):int(t_stim_end*len(T)/seconds), i] = params_external_input['v_e'] * (1 + A * np.exp(eta * (np.cos(np.deg2rad(angle_i) - np.deg2rad(stimuli_orientation)) - 1)))
    spike_trains = generate_spike_train(Ne, Ni, v, T, delta_T)

    #simulation itself
    states, firings, synaptic_input, synaptic_inputs, background_input, conductances, background_conductance = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, spike_trains, serotonin_weights, params_network)
    synpaptic_inputs[mu] = np.mean(synaptic_input[int(t_stim_end * 1000 / delta_T):, 100])
    nmda_synaptic_inputs[mu] = np.mean(synaptic_inputs[int(t_stim_end * 1000 / delta_T):, 100, 1])

In [ ]:
for mu in mus:
    if mu != 1:
        print(f"For mu = {mu}, synaptic input is {synpaptic_inputs[mu]/synpaptic_inputs[1] * 100:.02f}% of normal condition synaptic input")
        print(f"For mu = {mu}, NMDA synaptic input is {nmda_synaptic_inputs[mu]/nmda_synaptic_inputs[1] * 100:.02f}% of normal condition NMDA synaptic input")
        print()